This notebook
- resamples the worlpop 30' grid to 18'', the CMF downscaled flood depth grid resolution
- converts the 18' worldpop geoptif to CMF binary files for each highres area

In [ ]:
import rasterio
from rasterio.transform import array_bounds, from_bounds, from_origin
from rasterio.warp import Resampling
from rasterio.crs import CRS
from os.path import join, isfile
import numpy as np
import math
from numba import vectorize, njit
import pandas as pd

In [ ]:
def affine_to_coords(affine, shape):
    """Returs a raster axis with pixel center coordinates based on the affine.

    Parameters
    ----------
    affine : affine transform
        Two dimensional affine transform for 2D linear mapping
    shape : tuple of int
        The height, width  of the raster.

    Returns
    -------
    x, y coordinate arrays : tuple of ndarray of float
    """
    height, width = shape
    x_coords, _ = affine * (np.arange(width) + 0.5, np.zeros(width) + 0.5)
    _, y_coords = affine * (np.zeros(height) + 0.5, np.arange(height) + 0.5)
    return x_coords, y_coords

def reggrid_area(lats, lons):
    """returns a the cell area for a regular grid with cell centres lats & lons [m2]"""
    xres = np.abs(np.mean(np.diff(lons)))
    yres = np.abs(np.mean(np.diff(lats)))
    return cellarea(lats, xres, yres)[:,None]*np.ones((lats.size,lons.size), dtype=lats.dtype)

@vectorize([
    "float64(float64,float64,float64)", 
    "float32(float32,float32,float32)"
    ])
def cellarea(lat, xres, yres):
    """returns the area of cell with a given resolution (resx,resy) at a given cell center latitude [2]"""
    _R = 6371e3 # Radius of earth in m. Use 3956e3 for miles
    l1 = math.radians(lat-abs(yres)/2.)
    l2 = math.radians(lat+abs(yres)/2.)
    dx = math.radians(xres)
    return _R**2*dx*(math.sin(l2) - math.sin(l1))

## resample worldpop grid

In [ ]:
# IN
wp_fn = r'/home/dirk/datasets/WorldPop/ppp_2010_1km_Aggregated.tif'
with rasterio.open(wp_fn, 'r') as src:
    print(src.profile)

In [ ]:
# src_tot_pop = 0
# with rasterio.open(wp_fn, 'r') as src:
#     for ji, wdw in src.block_windows(1):
#         pop_30sec = src.read(1, window=wdw)
#         src_tot_pop += np.nansum(pop_30sec[pop_30sec!=src.nodata])
# src_tot_pop
src_tot_pop = 6812530928.509598
dst_tot_pop_nearest = 6810460454.075542
dst_tot_pop_bilinear = 6812976189.401252
err_bil = np.abs(1-dst_tot_pop_bilinear/src_tot_pop)*100  
err_nn = np.abs(1-dst_tot_pop_nearest/src_tot_pop)*100
err_bil, err_nn, dst_tot_pop_bilinear-src_tot_pop

In [ ]:
# out
xmin, ymin, xmax, ymax = -180, -72, 180, 84
xsize, ysize = 0.005, 0.005
blockxsize, blockysize = 400, 400
wp2_fn = r'/home/dirk/datasets/WorldPop/ppp_2010_18arcsec_Aggregated_nearest.tif'

tot_pop = 0
# ppp to density 
wpd_fn = wp_fn.replace('.tif', '_density.tif')
with rasterio.open(wp_fn, 'r') as src:
    prof = src.profile.copy()
    prof.update(
        transform = from_origin(xmin, ymax, xsize, ysize),
        width = int((xmax-xmin) / xsize),
        height = int((ymax-ymin) / xsize),
        blockxsize = blockxsize,
        blockysize = blockysize,
    )
    with rasterio.open(wp2_fn, 'w', **prof) as dst:
        for ji, dst_wdw in dst.block_windows(1):
#             if not np.all(ji == (10, 10)): continue
            
            # get window properties dst (18sec)
            dst_transform = dst.window_transform(dst_wdw)
            dst_shape = dst_wdw.height, dst_wdw.width
            dst_bounds = array_bounds(*dst_shape, transform=dst_transform)
            
            # get src window  (30sec)
            src_wdw = rasterio.windows.round_window_to_full_blocks(
                window=rasterio.windows.from_bounds(*dst_bounds, transform=src.transform), 
                block_shapes=src.block_shapes
            )

            # read pop
            pop_30sec = src.read(1, window=src_wdw)
            
            # convert to density
            src_transform = src.window_transform(src_wdw)
            src_bounds = array_bounds(*pop_30sec.shape, transform=src_transform)
            src_lons, src_lats = affine_to_coords(src_transform, pop_30sec.shape[1], pop_30sec.shape[0])
            area_30sec = reggrid_area(src_lats, src_lons) #km
            dens_30sec = np.where(pop_30sec != src.nodata, pop_30sec / area_30sec, src.nodata)
            assert np.all(np.isfinite(dens_30sec))
            
            # resample
            dens_18sec = np.ones(dst_shape)*dst.nodata
            rasterio.warp.reproject(
                source=dens_30sec,
                destination=dens_18sec,
                resampling=Resampling.nearest,
                src_transform=src_transform,
                src_crs=src.crs,
                src_nodata=src.nodata,
                dst_transform=dst_transform,
                dst_height=dst_wdw.height, 
                dst_width=dst_wdw.width,
                dst_crs=dst.crs,
                dst_nodata=dst.nodata
            )
            assert np.all(np.isfinite(dens_18sec))
            # to pop per pixel
            dst_lons, dst_lats = affine_to_coords(dst_transform, dst_wdw.width, dst_wdw.height)
            area_18sec = reggrid_area(dst_lats, dst_lons)
            pop_18sec = np.where(dens_18sec!=dst.nodata, dens_18sec * area_18sec, dst.nodata)
            tot_pop += pop_18sec[pop_18sec!=src.nodata].sum()
            # write
            dst.write(pop_18sec.astype(np.float32), window=dst_wdw, indexes=1)
print(tot_pop)
print(tot_pop/src_tot_pop)

# write to binary files for fortran impact routine

In [ ]:
mdir = r'/home/dirk/models/CaMa-Flood_v3.6.2/map/global_15min'
fn_regions = join(mdir,'hires', 'location.txt')
regions = pd.read_csv(fn_regions, delim_whitespace=True, index_col=0).T \
            .set_index('area').astype(float).to_dict(orient='index')

In [ ]:
import xarray as xr 

wp2_fn = r'/home/dirk/datasets/WorldPop/ppp_2010_18arcsec_Aggregated_bilinear.tif'

for area in regions:
    fn = join(mdir, 'hires', f'{area}.catmxy.tif')
    fn_out_bin = join(mdir, 'hires', f'{area}.worldpop')
    if isfile(fn_out_bin): continue
    print(area)
    with rasterio.open(fn, 'r') as like:
        bounds = like.bounds
        # xarray to deal with si2 which passes -180 line
        ds = xr.open_rasterio(wp2_fn, chunks={'x':400, 'y':400}).drop('band').squeeze()
        mv = ds.attrs['nodatavals'][0]
        w, s, e, n = bounds
        if e > 180 or w < -180:
            lon_org = np.copy(ds['x'].values)
            if e > 180:
                ds['x'] = xr.Variable('x', np.where(lon_org < 0, lon_org + 360, lon_org))
            else:
                ds['x'] = xr.Variable('x', np.where(lon_org > 0, lon_org - 360, lon_org))
            ds = ds.sortby('x')
        pop = ds.sel({'x':slice(w, e), 'y':slice(n, s)}).values
        ds.close()
        assert np.all(pop.shape == like.shape)
        np.where(pop!=mv, pop, 0).astype(np.float32).tofile(fn_out_bin) # mv zeros

## sum exposure at unit catchment level

In [ ]:
@njit
def sum_exp_ucat(catmx, catmy, flddif, exp, lecz, elevtn, exp_agg, exp_agg_lecz):
    """sum exposure at unit catchment level"""
    ny, nx = catmx.shape
    NY, NX = exp_agg.shape
    for iy in range(ny):
        for ix in range(nx):
            if catmx[iy, ix]>0:
                r, c = catmy[iy, ix]-1, catmx[iy, ix]-1
#                 assert r>=0 and r<NY and c>=0 and c<NX
                exp0 = exp[iy, ix]
                if exp0 <= 0: continue
                exp_agg[r,c] += exp0
                if elevtn[r,c] + flddif[iy, ix] < 10 and lecz[r,c]:
                    exp_agg_lecz[r,c] += exp0
    return exp_agg, exp_agg_lecz

In [ ]:

nodata=-9999.
bbox = -180., -90., 180., 90.
res = 0.25
NX, NY = 1440, 720
profile = {
    'driver': 'GTiff', 
    'dtype': 'float32', 
    'nodata': nodata, 
    'width': ((bbox[2]-bbox[0])/res), 
    'height': ((bbox[3]-bbox[1])/res), 
    'count': 1, 
    'crs': CRS.from_epsg(4326), 
    'transform': from_origin(bbox[0], bbox[3], res, res),
    'compress': 'lzw', 
    'interleave': 'band'
}

In [ ]:
map_dir = r'/home/dirk/models/cama-flood_bmi_v3.6.2_nc/map/global_15min'

fn_regions = join(map_dir, 'hires', 'location.txt')
regions = pd.read_csv(fn_regions, delim_whitespace=True, index_col=0).T \
            .set_index('area').astype(float).to_dict(orient='index')

# high res maps
fn_flddif = join(map_dir, 'hires', '{}.flddif').format
fn_catmxy = join(map_dir, 'hires', '{}.catmxy').format
fn_exp = join(map_dir, 'hires', '{}.worldpop').format

# model res maps
fn_elevtn = join(map_dir, 'elevtn.bin')
with open(fn_elevtn, 'r') as fid:
    elevtn = np.fromfile(fid, 'f4').reshape(NY, NX)
fn_lecz = join(map_dir, 'lecz_10m_basin.tif')
with rasterio.open(fn_lecz, 'r') as src:
    lecz = np.logical_and(src.read(1)>0, src.read(1)!=src.nodata)

# out maps
fn_exp_agg = join(map_dir, 'worldpop.tif')
fn_exp_agg_lecz = join(map_dir, 'lecz_worldpop.tif')
exp_agg = np.zeros((NY, NX))
exp_agg_lecz = np.zeros((NY, NX))

for area in regions.keys():
    ny, nx = int(regions[area]['ny']), int(regions[area]['nx'])
    with open(fn_flddif(area), 'r') as fid:
        flddif = np.fromfile(fid, 'f4').reshape(ny, nx)
    with open(fn_catmxy(area), 'r') as fid:
        catmx, catmy = np.fromfile(fid, 'i2').reshape(2, ny, nx)
    with open(fn_exp(area), 'r') as fid:
        exp = np.fromfile(fid, 'f4').reshape(ny, nx)
        
    exp_agg, exp_agg_lecz = sum_exp_ucat(catmx, catmy, flddif, exp, lecz, elevtn, exp_agg, exp_agg_lecz)
    print(area, exp_agg[exp_agg!=-9999].sum()/1e6)

with rasterio.open(fn_exp_agg, 'w', **profile) as dst:
    exp_agg[elevtn==elevtn[0,0]] = nodata
    dst.write(exp_agg.astype(np.float32), 1)
with rasterio.open(fn_exp_agg_lecz, 'w', **profile) as dst:
    exp_agg_lecz[elevtn==elevtn[0,0]] = nodata
    dst.write(exp_agg_lecz.astype(np.float32), 1)

## sum exposure at unit catchment level

In [ ]:
@njit
def sum_exp_ucat(catmx, catmy, flddif, exp, lecz, elevtn, exp_agg, exp_agg_lecz):
    """sum exposure at unit catchment level"""
    ny, nx = catmx.shape
    NY, NX = exp_agg.shape
    for iy in range(ny):
        for ix in range(nx):
            if catmx[iy, ix]>0:
                r, c = catmy[iy, ix]-1, catmx[iy, ix]-1
#                 assert r>=0 and r<NY and c>=0 and c<NX
                exp0 = exp[iy, ix]
                if exp0 <= 0: continue
                exp_agg[r,c] += exp0
                if elevtn[r,c] + flddif[iy, ix] < 10 and lecz[r,c]:
                    exp_agg_lecz[r,c] += exp0
    return exp_agg, exp_agg_lecz

In [ ]:

nodata=-9999.
bbox = -180., -90., 180., 90.
res = 0.25
NX, NY = 1440, 720
profile = {
    'driver': 'GTiff', 
    'dtype': 'float32', 
    'nodata': nodata, 
    'width': ((bbox[2]-bbox[0])/res), 
    'height': ((bbox[3]-bbox[1])/res), 
    'count': 1, 
    'crs': CRS.from_epsg(4326), 
    'transform': from_origin(bbox[0], bbox[3], res, res),
    'compress': 'lzw', 
    'interleave': 'band'
}

In [ ]:
map_dir = r'/home/dirk/models/cama-flood_bmi_v3.6.2_nc/map/global_15min'

fn_regions = join(map_dir, 'hires', 'location.txt')
regions = pd.read_csv(fn_regions, delim_whitespace=True, index_col=0).T \
            .set_index('area').astype(float).to_dict(orient='index')

# high res maps
fn_flddif = join(map_dir, 'hires', '{}.flddif').format
fn_catmxy = join(map_dir, 'hires', '{}.catmxy').format
fn_exp = join(map_dir, 'hires', '{}.worldpop').format

# model res maps
fn_elevtn = join(map_dir, 'elevtn.bin')
with open(fn_elevtn, 'r') as fid:
    elevtn = np.fromfile(fid, 'f4').reshape(NY, NX)
fn_lecz = join(map_dir, 'lecz_10m_basin.tif')
with rasterio.open(fn_lecz, 'r') as src:
    lecz = np.logical_and(src.read(1)>0, src.read(1)!=src.nodata)

# out maps
fn_exp_agg = join(map_dir, 'worldpop.tif')
fn_exp_agg_lecz = join(map_dir, 'lecz_worldpop.tif')
exp_agg = np.zeros((NY, NX))
exp_agg_lecz = np.zeros((NY, NX))

for area in regions.keys():
    ny, nx = int(regions[area]['ny']), int(regions[area]['nx'])
    with open(fn_flddif(area), 'r') as fid:
        flddif = np.fromfile(fid, 'f4').reshape(ny, nx)
    with open(fn_catmxy(area), 'r') as fid:
        catmx, catmy = np.fromfile(fid, 'i2').reshape(2, ny, nx)
    with open(fn_exp(area), 'r') as fid:
        exp = np.fromfile(fid, 'f4').reshape(ny, nx)
        
    exp_agg, exp_agg_lecz = sum_exp_ucat(catmx, catmy, flddif, exp, lecz, elevtn, exp_agg, exp_agg_lecz)
    print(area, exp_agg[exp_agg!=-9999].sum()/1e6)

with rasterio.open(fn_exp_agg, 'w', **profile) as dst:
    exp_agg[elevtn==elevtn[0,0]] = nodata
    dst.write(exp_agg.astype(np.float32), 1)
with rasterio.open(fn_exp_agg_lecz, 'w', **profile) as dst:
    exp_agg_lecz[elevtn==elevtn[0,0]] = nodata
    dst.write(exp_agg_lecz.astype(np.float32), 1)

In [ ]:
ds_out = xr.open_dataset(join(ddir, r'rivmth_swe_am_ci.nc')).sel(scen=['surge', 'seas', 'tide']).load()
ds_out.close()
ds_out = ds_out.rename({'WSE_am': 'annual_maxima', 'WSE_ev': 'extreme_values', 'WSE_ev_ci': 'extreme_values_ci'})
ds_out['annual_maxima'].attrs = dict(
    description='simulated annual maxima of water surface elevation at the river mouth',
    unit='m+EGM96',
    long_name='water_surface_elevation'
)
ds_out['extreme_values'].attrs = dict(
    description='extreme values of water surface elevation at the river mouth based on Gubmel distribion of annual maxima',
    unit='m+EGM96',
    long_name='water_surface_elevation'
)
ds_out['extreme_values_ci'].attrs = dict(
    description='confidence intervals around extreme values of water surface elevation',
    unit='m+EGM96',
    long_name='water_surface_elevation'
)
ds_out['params'].attrs = dict(
    description='parameters of Gumbel extreme value distibution',
    unit='-',
)
ds_out.attrs.update(
    institution = 'Institute for Environmental Studies (IVM) - Vrije Universiteit Amsterdam',
    author = 'Dirk Eilander (dirk.eilander@vu.nl)',
    date_created = str(datetime.now().date()),
    history = f'created using s03-rivmth_ev; xarray v{xr.__version__}',
)
ds_out.to_netcdf(join(ddir, r'rivmth_wse_ev.nc'))
ds_out.close()